In [1]:
import numpy as np
import pandas as pd
import matplotlib 
import seaborn as sns
from matplotlib import pyplot as plt

dataset=pd.read_csv('winequality-white.csv')
pd.options.mode.chained_assignment = None 

In [2]:
dataset['rating']='NaN'


In [3]:
dataset['rating'][dataset.quality<=3]=0  # Bad wine
dataset['rating'][dataset.quality>=7]=1 # Average Wine
dataset['rating'][(dataset.quality>=4) & (dataset.quality<=6)]=2  # Good Wine
dataset['rating'].astype(int)

0       2
1       2
2       2
3       2
4       2
       ..
4893    2
4894    2
4895    2
4896    1
4897    2
Name: rating, Length: 4898, dtype: int32

In [4]:
dataset.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,rating
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,2
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,2
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,2
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,2
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,2


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset.iloc[:, 1:11].values, dataset.rating.values, test_size = 0.2, random_state = 0)

In [6]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
y_train=y_train.astype('int')
y_test=y_test.astype('int')


In [7]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', max_depth=10,random_state = 0)
classifier.fit(X_train, y_train)



# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm_test = confusion_matrix(y_test, y_pred)
print('******Test set confusion matrix************')
print(cm_test)




******Test set confusion matrix************
[[  0   0   9]
 [  0  87 129]
 [  0  30 725]]


In [8]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [5, 10, 15,20], 'max_features': [1,2,3,4,5], 'max_depth':[5,10,15],'min_samples_split':[2,5,10],'min_samples_leaf':[1,5,10]},
    {'bootstrap': [False], 'n_estimators': [5, 10, 20], 'max_features': [1,2,3,4,5],'max_depth':[5,10,15],'min_samples_split':[2,5,10],'min_samples_leaf':[1,5,10]}
  ]

grid_search = GridSearchCV(estimator = classifier,
                           param_grid = param_grid,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

In [9]:
print( grid_search.best_score_)
print(grid_search.best_params_)

0.8736600306278713
{'bootstrap': False, 'max_depth': 15, 'max_features': 2, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 20}


In [10]:
from sklearn.model_selection import cross_val_score
accuracies_train = cross_val_score(estimator = grid_search.best_estimator_, X = X_train, y = y_train, cv = 10)
accuracies_test = cross_val_score(estimator = grid_search.best_estimator_, X = X_test, y = y_test, cv = 10)
print('Training accuracy mean',accuracies_train.mean())
print('Training accuracy std',accuracies_train.std())
print('Testing accuracy mean',accuracies_test.mean())
print('Testing accuracy std',accuracies_test.std())

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Training accuracy mean 0.8736623255207794
Training accuracy std 0.007378414045959058
Testing accuracy mean 0.8102950196792242
Testing accuracy std 0.028612968470409027


In [9]:
import tensorflow as tf
from tensorflow import keras

In [10]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(20, activation="relu"))
model.add(keras.layers.Dense(20, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [11]:
model.compile( loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [12]:
X_valid=X_train[:500]
y_valid=y_train[:500]

In [13]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
history = model.fit(X_train, y_train, epochs=100,validation_data=(X_valid, y_valid)) 

Train on 3918 samples, validate on 500 samples
Epoch 1/100
3918/3918 [==============================] - 1s 262us/sample - loss: 1.4429 - accuracy: 0.6192 - val_loss: 0.8484 - val_accuracy: 0.8040
Epoch 2/100
3918/3918 [==============================] - 0s 44us/sample - loss: 0.7498 - accuracy: 0.7818 - val_loss: 0.5863 - val_accuracy: 0.8040
Epoch 3/100
3918/3918 [==============================] - 0s 47us/sample - loss: 0.5812 - accuracy: 0.7820 - val_loss: 0.4996 - val_accuracy: 0.8040
Epoch 4/100
3918/3918 [==============================] - 0s 42us/sample - loss: 0.5211 - accuracy: 0.7836 - val_loss: 0.4655 - val_accuracy: 0.7820
Epoch 5/100
3918/3918 [==============================] - 0s 51us/sample - loss: 0.4932 - accuracy: 0.7879 - val_loss: 0.4528 - val_accuracy: 0.7840
Epoch 6/100
3918/3918 [==============================] - 0s 44us/sample - loss: 0.4772 - accuracy: 0.7917 - val_loss: 0.4378 - val_accuracy: 0.7860
Epoch 7/100
3918/3918 [==============================] - 0s 45us

Epoch 56/100
3918/3918 [==============================] - 0s 49us/sample - loss: 0.3896 - accuracy: 0.8244 - val_loss: 0.3946 - val_accuracy: 0.8140
Epoch 57/100
3918/3918 [==============================] - 0s 43us/sample - loss: 0.3895 - accuracy: 0.8229 - val_loss: 0.3925 - val_accuracy: 0.8280
Epoch 58/100
3918/3918 [==============================] - 0s 44us/sample - loss: 0.3893 - accuracy: 0.8234 - val_loss: 0.3921 - val_accuracy: 0.8160
Epoch 59/100
3918/3918 [==============================] - 0s 43us/sample - loss: 0.3883 - accuracy: 0.8249 - val_loss: 0.3910 - val_accuracy: 0.8260
Epoch 60/100
3918/3918 [==============================] - 0s 43us/sample - loss: 0.3880 - accuracy: 0.8262 - val_loss: 0.3909 - val_accuracy: 0.8240
Epoch 61/100
3918/3918 [==============================] - 0s 43us/sample - loss: 0.3874 - accuracy: 0.8229 - val_loss: 0.3939 - val_accuracy: 0.8040
Epoch 62/100
3918/3918 [==============================] - 0s 44us/sample - loss: 0.3862 - accuracy: 0.8231